In [ ]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

In [ ]:
import uchimata
import numpy as np
import pyarrow as pa

In [ ]:
# Loading from local arrow file
with pa.memory_map('../../sample_data/test.arrow', 'r') as source:
    table_local_file = pa.ipc.open_stream(source).read_all()
    
output_stream = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream, table_local_file.schema) as writer:
    writer.write_table(table_local_file)

N = table_local_file.num_rows
numbers = list(range(1, N+1))

vc = {
    "color": {
        "values": numbers,
        "min": 1,
        "max": N+1,
        "colorScale": "Spectral"
    }, 
    "scale": 0.02, "links": True, "mark": "sphere"
}

# Get the bytes from the output stream
table_bytes = output_stream.getvalue().to_pybytes()
print(table_local_file)
uchimata.Widget(table_bytes, viewconfig=vc)

In [ ]:
# Loading from local arrow file
# '/Users/davidkouril/projects/chromospace/data/model/model.arrow'
with pa.memory_map('/Users/davidkouril/projects/chromospace/data/dros.3.arrow', 'r') as source:
    table_local_file = pa.ipc.open_stream(source).read_all()
    
# arr = loaded_arrays[0]
# print(f"{arr[0]} .. {arr[-1]}")
print(table_local_file)

In [ ]:
# Make an Arrow Table from the x, y, z coordinate arrays
# table = pa.Table.from_arrays(decons_data, names=["x", "y", "z"])

# '/Users/davidkouril/projects/chromospace/scripts/out/stevens-cell1-test.arrow'
with pa.memory_map('/Users/davidkouril/projects/chromospace/scripts/out/stevens-model.arrow', 'r') as source:
    table_local_file = pa.ipc.open_stream(source).read_all()

print(table_local_file)
# Convert the Table to bytes
output_stream = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream, table_local_file.schema) as writer:
    writer.write_table(table_local_file)

N = 200
numbers = list(range(1, N+1))

vc = {
    "color": "purple", 
    "scale": 0.001, "links": True, "mark": "sphere"
}

# Get the bytes from the output stream
table_bytes = output_stream.getvalue().to_pybytes()
# Show the structure via uchimata widget
uchimata.Widget(table_bytes, viewconfig=vc)

In [ ]:
import pyarrow.feather as feather
from pathlib import Path

table = feather.read_feather('/Users/davidkouril/Downloads/model.arrow')
# print(table)

# file = Path('/Users/davidkouril/Downloads/model.arrow')
file = Path('/Users/davidkouril/projects/spatial-genome-data/sbif-2021/out/dros.3.arrow')
# file = Path('/Users/davidkouril/projects/spatial-genome-data/stevens-2017/out/GSM2219497_Cell_1_model_1.arrow')
file_as_bytes = file.read_bytes()

uchimata.Widget(file_as_bytes)

## Loading a local Arrow file

## Passing a numpy array

In [ ]:
import random

def make_random_3D_chromatin_structure(n):
    position = np.array([0.0, 0.0, 0.0])
    positions = [position.copy()]
    x_arr = []
    y_arr = []
    z_arr = []
    for _ in range(n):
        step = np.random.choice([-1.0, 0.0, 1.0], size=3)  # Randomly choose to move left, right, up, down, forward, or backward
        position += step
        positions.append(position.copy())
    return positions

def deconstruct_into_column_arrays(nparr):
    x_arr = []
    y_arr = []
    z_arr = []
    for row in nparr:
        x_arr.append(row[0])
        y_arr.append(row[1])
        z_arr.append(row[2])
        
    return [x_arr, y_arr, z_arr]

In [ ]:
## Use case 1: Passing raw numpy array
test_data = make_random_3D_chromatin_structure(n=100)
data = np.array(test_data)

decons_data = deconstruct_into_column_arrays(data)
# decons_data

In [ ]:
# Make an Arrow Table from the x, y, z coordinate arrays
table = pa.Table.from_arrays(decons_data, names=["x", "y", "z"])

# Convert the Table to bytes
output_stream = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream, table.schema) as writer:
    writer.write_table(table)

# Get the bytes from the output stream
table_bytes = output_stream.getvalue().to_pybytes()
# table_bytes

In [ ]:
# Show the structure via uchimata widget
vc = {"color": "#6600ff", "scale": 0.02, "links": True, "mark": "sphere"}

uchimata.Widget(table_bytes, viewconfig=vc)

## Parsing an XYZ string

In [ ]:
# ## Use case 2: Passing XYZ string
test_string = """6
sample = 0
CA	1.101124	0.547027	-2.299305
CA	-0.919687	1.275041	-3.201912
CA	-0.974526	1.144577	-4.491504
CA	-0.881394	-0.011930	-4.496625
CA	-0.455539	-2.518781	-2.500660
CA	-0.626368	-2.086400	-3.427676
"""

def parseXYZ(content):
    x_array = []
    y_array = []
    z_array = []
    delimiter='\t'
    lines = content.splitlines()
    # lines
    lineNumber = 0
    firstLine = lines[0]
    lineNum = int(firstLine)
    for line in lines:
        if lineNumber < 2:
            lineNumber = lineNumber + 1
            continue
        if lineNumber > lineNum:
            break
        lineNumber = lineNumber + 1
        tokens = line.split(delimiter)
        if len(tokens) < 4:
            continue
        
        x = float(tokens[1])
        y = float(tokens[2])
        z = float(tokens[3])
        x_array.append(x)
        y_array.append(y)
        z_array.append(z)

    return [x_array, y_array, z_array]

# x_array
res = parseXYZ(test_string)
x_array = res[0]
y_array = res[1]
z_array = res[2]
# Make an Arrow Table from the x, y, z coordinate arrays
table2 = pa.Table.from_arrays([x_array, y_array, z_array], names=["x", "y", "z"])

# Convert the Table to bytes
output_stream2 = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream2, table2.schema) as writer:
    writer.write_table(table2)

# Get the bytes from the output stream
table2_bytes = output_stream2.getvalue().to_pybytes()

# uchimata.Widget(structure_string=test_string, delimiter="\t")
uchimata.Widget(table2_bytes)

In [ ]:
# ## Use case 3: Passing XYZ file path
# import pathlib
# test_path = pathlib.Path("./sample_data/test.xyz")

with open('../../sample_data/test.xyz', 'r') as file:
    content = file.read()
    # print(content)
    res = parseXYZ(content)
    x_array = res[0]
    y_array = res[1]
    z_array = res[2]
    # Make an Arrow Table from the x, y, z coordinate arrays
    table3 = pa.Table.from_arrays([x_array, y_array, z_array], names=["x", "y", "z"])
    
    # Convert the Table to bytes
    output_stream3 = pa.BufferOutputStream()
    with pa.ipc.RecordBatchStreamWriter(output_stream3, table3.schema) as writer:
        writer.write_table(table3)
    
    # Get the bytes from the output stream
    table3_bytes = output_stream3.getvalue().to_pybytes()
    

# supplying an array of values for the color/size

N = 200
numbers = list(range(1, N+1))

vc = {
    "color": {
        "values": numbers,
        "min": 1,
        "max": 100,
        "colorScale": "Spectral"
    }, 
    "scale": 0.02, "links": True, "mark": "sphere"
}

uchimata.Widget(table3_bytes, viewconfig=vc)